In [6]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.9 MB/s eta 0:00:00


In [2]:
!unzip archive.zip

Archive:  archive.zip
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024468.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024470.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024707.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024763.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024771.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024800.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024913.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024948.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0025780.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0025953.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0025957.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0025992.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0026149.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0026171.jpg  
  inflating: Split_smol/train/Actinic ke

In [4]:
# ============================================================
# 피부질환 분류 ResNeXt 학습 코드 (결과 fig/accuracy_curve.png 저장)
# ============================================================

import torch
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import os

# ---------------------------
# 1. 경로 및 하이퍼파라미터
# ---------------------------
data_root  = "/content/Split_smol"   # 기준 루트
train_dir  = os.path.join(data_root, "train")
val_dir    = os.path.join(data_root, "val")

batch_size = 32
num_epochs = 20
learning_rate = 0.001
momentum = 0.9

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

# ---------------------------
# 2. 데이터셋 & 데이터로더
# ---------------------------
transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(root=train_dir, transform=transform)
val_dataset   = ImageFolder(root=val_dir,   transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)

print(f"[INFO] Classes: {train_dataset.classes}")
num_classes = len(train_dataset.classes)

# ---------------------------
# 3. 모델 정의 (ResNeXt50)
# ---------------------------
try:
    weights = models.ResNeXt50_32X4D_Weights.DEFAULT
    model = models.resnext50_32x4d(weights=weights)
except:
    model = models.resnext50_32x4d(pretrained=True)

model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# ---------------------------
# 4. Loss & Optimizer
# ---------------------------
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

# ---------------------------
# 5. Accuracy 계산 함수
# ---------------------------
def compute_accuracy(outputs, labels):
    preds = torch.argmax(outputs, dim=1)
    return (preds == labels).sum().item()

# ---------------------------
# 6. 학습 루프
# ---------------------------
train_acc_history = []
val_acc_history = []

for epoch in range(1, num_epochs+1):
    # ---- Train ----
    model.train()
    running_correct = 0
    running_total   = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_correct += compute_accuracy(outputs, labels)
        running_total   += labels.size(0)

    train_acc = 100 * running_correct / running_total
    train_acc_history.append(train_acc)

    # ---- Validation ----
    model.eval()
    val_correct = 0
    val_total   = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_correct += compute_accuracy(outputs, labels)
            val_total   += labels.size(0)

    val_acc = 100 * val_correct / val_total
    val_acc_history.append(val_acc)

    # ---- 로그 출력 ----
    print(f"Epoch [{epoch}/{num_epochs}] "
          f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

# ---------------------------
# 7. Accuracy 그래프 저장 (/fig)
# ---------------------------
os.makedirs("fig", exist_ok=True)  # fig 폴더 없으면 생성

plt.figure(figsize=(8,6))
plt.plot(range(1, num_epochs+1), train_acc_history, label="Train Acc")
plt.plot(range(1, num_epochs+1), val_acc_history, label="Val Acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Training & Validation Accuracy")
plt.legend()
plt.grid(True)

save_path = os.path.join("fig", "accuracy_curve.png")
plt.savefig(save_path, dpi=150)
plt.close()

print(f"[INFO] Training Finished. Accuracy curve saved to {save_path}")


[INFO] Using device: cuda
[INFO] Classes: ['Actinic keratosis', 'Atopic Dermatitis', 'Benign keratosis', 'Dermatofibroma', 'Melanocytic nevus', 'Melanoma', 'Squamous cell carcinoma', 'Tinea Ringworm Candidiasis', 'Vascular lesion']
Epoch [1/20] Train Acc: 18.36% | Val Acc: 32.60%
Epoch [2/20] Train Acc: 48.21% | Val Acc: 55.25%
Epoch [3/20] Train Acc: 67.86% | Val Acc: 62.98%
Epoch [4/20] Train Acc: 76.04% | Val Acc: 66.30%
Epoch [5/20] Train Acc: 81.78% | Val Acc: 70.17%
Epoch [6/20] Train Acc: 84.22% | Val Acc: 72.38%
Epoch [7/20] Train Acc: 85.65% | Val Acc: 76.80%
Epoch [8/20] Train Acc: 91.68% | Val Acc: 79.56%
Epoch [9/20] Train Acc: 91.39% | Val Acc: 78.45%
Epoch [10/20] Train Acc: 94.55% | Val Acc: 76.24%
Epoch [11/20] Train Acc: 94.84% | Val Acc: 77.90%
Epoch [12/20] Train Acc: 96.13% | Val Acc: 78.45%
Epoch [13/20] Train Acc: 97.70% | Val Acc: 80.66%
Epoch [14/20] Train Acc: 97.85% | Val Acc: 79.56%
Epoch [15/20] Train Acc: 98.42% | Val Acc: 79.01%
Epoch [16/20] Train Acc: 98

# 파일 업로드

In [ ]:
import boto3

# S3 클라이언트 생성
s3 = boto3.client(
    's3',
    aws_access_key_id='your_key',         # 본인 키
    aws_secret_access_key='your_secret',     # 본인 시크릿 키
    region_name='us-east-1'                  # 버지니아
)

# 파일 업로드
s3.upload_file(
    Filename='fig/accuracy_curve.png',                 # 로컬 파일명 (a.zip)
    Bucket='alpacom2ysk',              # 버킷 이름
    Key='fig/피부질환분류_결과/accuracy_curve.png'                # S3 내 저장 경로
)

print("업로드 완료")

업로드 완료
